# Utilities

Useful helpers

## TinychatAgent

`TinychatAgent.create` gives you a new agent with tinychat lexicon namespace
attached. If you don't pass an instance of atproto agent into create, it will
try to create and authenticate a test agent using `TEST_AGENT_SERVICE`,
`TEST_AGENT_IDENTIFIER`, `TEST_AGENT_PASSWORD` env vars. For example,

```
TEST_AGENT_SERVICE=https://bsky.social
TEST_AGENT_IDENTIFIER=foo
TEST_AGENT_PASSWORD=bar
```

In [ ]:
//| export

import { Agent, AtpAgent } from "@atproto/api";
import { ChatNS } from "tinychat/api/index.ts";

export class TinychatAgent {
  public chat: ChatNS;
  public agent: Agent;

  constructor(agent: Agent) {
    this.chat = new ChatNS(agent);
    this.agent = agent;
  }

  static async create(agent?: Agent | undefined): Promise<TinychatAgent> {
    if (agent) {
      return new TinychatAgent(agent);
    }

    const [service, identifier, password] = [
      Deno.env.get("TEST_AGENT_SERVICE"),
      Deno.env.get("TEST_AGENT_IDENTIFIER"),
      Deno.env.get("TEST_AGENT_PASSWORD"),
    ];

    if (!service || !identifier || !password) {
      throw new Error(
        "Missing TEST_AGENT_SERVICE, TEST_AGENT_IDENTIFIER, or TEST_AGENT_PASSWORD",
      );
    }

    const testAgent = new AtpAgent({ service });
    await testAgent.login({ identifier, password });

    return new TinychatAgent(testAgent);
  }
}

In [ ]:
import { assert } from "asserts";

Deno.test("make sure test client works", async () => {
  const ta = await TinychatAgent.create();
  const { records } = await ta.chat.tinychat.server.list({
    repo: ta.agent.assertDid,
  });
  assert(typeof records.length !== "undefined");
});

## Unslopify imports and exports

atproto codegens generate modules with sloppy imports like this:

```ts
import * as ChatTinychatActorProfile from "./types/chat/tinychat/actor/profile";
import * as ChatTinychatServer from "./types/chat/tinychat/server";
```

We need to convert this to something like this

```ts
import * as ChatTinychatActorProfile from "./types/chat/tinychat/actor/profile.ts";
import * as ChatTinychatServer from "./types/chat/tinychat/server.ts";
```

Let's create `processLine` to process one a line from ts module

In [ ]:
//| export

const processLine = (line: string): string => {
  if (!line.trim().match(/^import|export/ig)) {
    return line;
  }
  const module = line.split("from").pop()?.trim().replaceAll(/'|"|;/ig, "");
  if (!module || !module.startsWith(".") || module.endsWith(".ts")) {
    return line;
  }
  return line.replace(module!, `${module}.ts`);
};

In [ ]:
import { assertEquals } from "asserts";

Deno.test("processLine", () => {
  assertEquals(
    processLine(
      `export * as ComAtprotoTempRequestPhoneVerification from "./types/com/atproto/temp/requestPhoneVerification";`,
    ),
    `export * as ComAtprotoTempRequestPhoneVerification from "./types/com/atproto/temp/requestPhoneVerification.ts";`,
  );
  assertEquals(
    processLine(
      `import * as ComAtprotoTempRequestPhoneVerification from "./types/com/atproto/temp/requestPhoneVerification";`,
    ),
    `import * as ComAtprotoTempRequestPhoneVerification from "./types/com/atproto/temp/requestPhoneVerification.ts";`,
  );
  assertEquals(processLine(`export class ChatNS {`), `export class ChatNS {`);
  assertEquals(
    processLine(
      `import * as ComAtprotoTempRequestPhoneVerification from "./types/com/atproto/temp/requestPhoneVerification.ts";`,
    ),
    `import * as ComAtprotoTempRequestPhoneVerification from "./types/com/atproto/temp/requestPhoneVerification.ts";`,
  );
});

`processFile` runs conversion of the whole file

In [ ]:
//| export

const processFile = async (file: string): Promise<string> => {
  const text = await Deno.readTextFile(file);
  const modifiedText = text.split("\n").map(processLine).join("\n");
  await Deno.writeTextFile(file, modifiedText);
  return modifiedText;
};


In [ ]:
Deno.test("processFile", async () => {
  const td = await Deno.makeTempDir({});
  await Deno.writeTextFile(
    `${td}/test.ts`,
    `
    export * as Foo from "./foo";
    import { bar } from "./bar";

    export class ChatNS {
      public foo: Foo;
      public bar: bar;
    }
  `,
  );
  await processFile(`${td}/test.ts`);
  assertEquals(
    await Deno.readTextFile(`${td}/test.ts`),
    `
    export * as Foo from "./foo.ts";
    import { bar } from "./bar.ts";

    export class ChatNS {
      public foo: Foo;
      public bar: bar;
    }
  `,
  );
});

`unslopifyModules` is the main function to recursively process modules in a
directory

In [ ]:
//| export

import { walk } from "jsr:@std/fs/walk";

export const unslopifyModules = async (dir: string) => {
  for await (const dirEntry of walk(dir, { exts: ["ts"] })) {
    await processFile(dirEntry.path);
  }
};